In [1]:
import os
from pandas import read_csv, DataFrame

file_dir = "TMD"

/tmp/ipykernel_16437/1887638657.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  from pandas import read_csv, DataFrame


In [1]:
from persistence import RipsComplexDeepMD
rips = RipsComplexDeepMD("TMD/IVDW_4/26")

In [2]:
from embeddings import PBoW
emb = PBoW(rips, )


KeyError: (0, 1)

In [1]:
from dataset import PBowDataset

dataset = PBowDataset()

In [4]:
dataset[[0, 1]]

/workspaces/TwistedMaterials/embeddings.py:82: RuntimeWarning: invalid value encountered in divide
  features[t, ...] = np.sign(section) * np.sqrt(np.abs(section)) / np.linalg.norm(section, axis = 0)
/workspaces/TwistedMaterials/embeddings.py:82: RuntimeWarning: invalid value encountered in divide
  features[t, ...] = np.sign(section) * np.sqrt(np.abs(section)) / np.linalg.norm(section, axis = 0)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [2]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size = 10)

In [3]:
for sample in loader:
    print(sample)
    break

KeyError: 'Filepath'

In [2]:
from utils import PrecomputeIVDW
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

pivdw = PrecomputeIVDW()

In [10]:
pivdw.total_dict.keys()

dict_keys(['box', 'energy', 'force', 'rips', 'type', 'virial'])

In [3]:
def compute_frame(name:str, save_path:str = "TMD/frames"):
    filename = f"{name}_frame.parquet"
    out_path = os.path.join(save_path, filename)
    if os.path.exists(out_path):
        print(f"{filename} exists. Loading now.")
        return pd.read_parquet(out_path, engine='pyarrow')

    print(f"{filename} doesn't exist. Creating now.")
    data = list(pivdw.total_dict[name][()].items())
    num_files = len(data)
    frames = []
    for n in range(num_files):
        curr = data[n]
        curr_frame = pd.DataFrame(curr[1].tolist())
        curr_frame['path'] = curr[0]
        path_column = curr_frame['path'].apply(lambda x: x.split("/")[1:])
        path_frame = pd.DataFrame(path_column.tolist(), columns = ['Folder', 'mol_num'])
        new_frame = pd.concat([path_frame, curr_frame], axis = 1).reset_index()
        new_frame = new_frame.rename(columns = {"index": "transition"})
        new_frame = new_frame.melt(id_vars=["path", "Folder", "mol_num", 'transition'], var_name=f"{name}_id", ignore_index=True)
        frames.append(new_frame)

    total_frame = pd.concat(frames, axis = 0).sort_values(['Folder', 'mol_num', 'transition', f'{name}_id']).reset_index(drop = True)

    # save data
    print(f"Saving {filename}. \n")
    total_frame.to_parquet(out_path, compression='gzip', index=False)
    return total_frame

In [1]:
def compute_rips_frame(hm:int, save_path:str = "TMD/frames"):
    filename = f"rips_{hm}_frame.parquet"
    out_path = os.path.join(save_path, filename)
    if os.path.exists(out_path):
        print(f"{filename} exists. Loading now.")
        return pd.read_parquet(out_path, engine='pyarrow')

    print(f"{filename} doesn't exist. Creating now.")
    data = list(pivdw.total_dict['rips'][()].items())
    num_files = len(data)
    total_frames = []
    for n in tqdm(range(num_files)):
        path, cell_data = data[n]
        hom_data_frames = []
        for pers_idx, hom_data in cell_data.items():
            curr_hom_data = pd.DataFrame(hom_data[hm].tolist(), columns=['birth', 'death']).reset_index()
            curr_hom_data = curr_hom_data.rename(columns={"index": "pers_id"})
            curr_hom_data['transition'] = pers_idx
            hom_data_frames.append(curr_hom_data)    
        concat_hom_data = pd.concat(hom_data_frames, axis = 0, ignore_index=True)
        concat_hom_data['path'] = path
        path_column = concat_hom_data['path'].apply(lambda x: x.split("/")[1:])
        path_frame = pd.DataFrame(path_column.tolist(), columns = ['Folder', 'mol_num'])
        concat_hom_data = pd.concat([path_frame, concat_hom_data], axis = 1)
        total_frames.append(concat_hom_data)
        
    total_frames_concat = pd.concat(total_frames, axis = 0, ignore_index=True)\
        .sort_values(['Folder', 'mol_num', "transition", "pers_id"])\
        .reset_index(drop = True)
    print(f"Saving {filename}.\n")
    total_frames_concat.to_parquet(out_path, compression="gzip", index=False)
    return total_frames_concat


In [3]:
energy_frame = compute_frame('energy')
box_frame = compute_frame('box')
force_frame = compute_frame('force')
type_frame = compute_frame('type')
virial_frame = compute_frame('virial')


force_frame.parquet doesn't exist. Creating now.
type_frame.parquet doesn't exist. Creating now.
virial_frame.parquet doesn't exist. Creating now.


In [3]:
rips_0_frame = compute_rips_frame(hm=0)
rips_1_frame = compute_rips_frame(hm=1)
rips_2_frame = compute_rips_frame(hm=2)

rips_0_frame.parquet doesn't exist. Creating now.


100%|██████████| 96/96 [02:09<00:00,  1.35s/it]


Saving rips_0_frame.parquet.

rips_1_frame.parquet doesn't exist. Creating now.


100%|██████████| 96/96 [02:05<00:00,  1.30s/it]


Saving rips_1_frame.parquet.

rips_2_frame.parquet doesn't exist. Creating now.


100%|██████████| 96/96 [01:58<00:00,  1.24s/it]


Saving rips_2_frame.parquet.



In [62]:
energy_frame

,path,Folder,mol_num,transition,energy_id,value
0,TMD/IVDW_10/0,IVDW_10,0,0,0,-342.687154
1,TMD/IVDW_10/0,IVDW_10,0,1,0,-342.684186
2,TMD/IVDW_10/0,IVDW_10,0,2,0,-342.675324
3,TMD/IVDW_10/0,IVDW_10,0,3,0,-342.660840
4,TMD/IVDW_10/0,IVDW_10,0,4,0,-342.641145
...,...,...,...,...,...,...
135995,TMD/IVDW_4/9,IVDW_4,9,1995,0,-326.547335
135996,TMD/IVDW_4/9,IVDW_4,9,1996,0,-326.724594
135997,TMD/IVDW_4/9,IVDW_4,9,1997,0,-326.910187
135998,TMD/IVDW_4/9,IVDW_4,9,1998,0,-327.097351


In [56]:
energy_frame = pd.DataFrame(energy_dict.items(), columns = ['path', 'energy'])
energy_frame['energy'] = energy_frame['energy'].apply(lambda x: x.flatten())
num_values = energy_frame.loc[1, 'energy'].shape[0]
energies = pd.DataFrame(energy_frame['energy'].tolist(), columns = list(range(num_values)))
energy_frame = energy_frame.drop("energy", axis = 1)
path_column = energy_frame['path'].apply(lambda x: x.split("/")[1:])
path_frame = pd.DataFrame(path_column.tolist(), columns = ['Folder', 'mol_num'])
path_frame['mol_num'] = pd.to_numeric(path_frame['mol_num'])
energy_frame = pd.concat([energy_frame, path_frame, energies], axis = 1)

energy_frame = energy_frame.melt(id_vars = ['path', 'Folder', 'mol_num'], var_name="energy_num", value_name="energy")
energy_frame = energy_frame.sort_values(['Folder', 'mol_num', 'energy_num']).reset_index(drop = True)
energy_frame

,path,Folder,mol_num,energy_num,energy
0,TMD/IVDW_10/0,IVDW_10,0,0,-342.687154
1,TMD/IVDW_10/0,IVDW_10,0,1,-342.684186
2,TMD/IVDW_10/0,IVDW_10,0,2,-342.675324
3,TMD/IVDW_10/0,IVDW_10,0,3,-342.660840
4,TMD/IVDW_10/0,IVDW_10,0,4,-342.641145
...,...,...,...,...,...
191995,TMD/IVDW_4/31,IVDW_4,31,1995,NaN
191996,TMD/IVDW_4/31,IVDW_4,31,1996,NaN
191997,TMD/IVDW_4/31,IVDW_4,31,1997,NaN
191998,TMD/IVDW_4/31,IVDW_4,31,1998,NaN
